In [1]:
import requests
import os 

from config import config

CHAT_PDF_API = config.CHAT_PDF_API

# Upload file

In [ ]:
files = [
    ('file', ('file', open('./data/hiscox.pdf', 'rb'), 'application/octet-stream'))
]
headers = {
    'x-api-key': f'{CHAT_PDF_API}'
}

response = requests.post(
    'https://api.chatpdf.com/v1/sources/add-file', headers=headers, files=files)

if response.status_code == 200:
    print('Source ID:', response.json()['sourceId'])
    source_id = response.json()['sourceId']
else:
    print('Status:', response.status_code)
    print('Error:', response.text)

# Ask Questions

In [ ]:
question = "What combined ratio did syndicate 33 report in 2022 and 2021"

In [ ]:
headers = {
    'x-api-key': f'{CHAT_PDF_API}',
    "Content-Type": "application/json",
}

data = {
    "referenceSources": True,
    'sourceId': f"{source_id}",
    'messages': [
        {
            'role': "user",
            'content': f"{question}",
        }
    ]
}

response = requests.post(
    'https://api.chatpdf.com/v1/chats/message', headers=headers, json=data)

if response.status_code == 200:
    print('Result:', response.json()['content'])
    print('References:', response.json()['references'])
else:
    print('Status:', response.status_code)
    print('Error:', response.text)

# Delete PDF

In [ ]:
headers = {
  'x-api-key': f'{CHAT_PDF_API}',
  'Content-Type': 'application/json',
}

data = {
  'sources': [f'{source_id}'],
}

try:
  response = requests.post(
    'https://api.chatpdf.com/v1/sources/delete', json=data, headers=headers)
  response.raise_for_status()
  print('Success - pdf deleted')
except requests.exceptions.RequestException as error:
  print('Error:', error)
  print('Response:', error.response.text)

# Custom Version ---

#### curl -O https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

In [10]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PDFPlumberLoader, PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from pdf2image import convert_from_path

## Directory Loader Version ------

In [6]:
loader = DirectoryLoader("./docs", glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load_and_split()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(documents)
len(texts)

604

### Create embeddings

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/Users/nickaristidou/.pyenv/versions/3.10.8/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
db = Chroma.from_documents(texts, embeddings, persist_directory="chroma")

### Create chain

In [13]:
model_path = "./ggml-gpt4all-j-v1.3-groovy.bin"
llm = GPT4All(model=model_path, backend="gptj", verbose=False)

Found model file at  ./ggml-gpt4all-j-v1.3-groovy.bin
gptj_model_load: loading model from './ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ...

objc[46009]: Class GGMLMetalClass is implemented in both /Users/nickaristidou/.pyenv/versions/3.10.8/envs/venv/lib/python3.10/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x2c1070208) and /Users/nickaristidou/.pyenv/versions/3.10.8/envs/venv/lib/python3.10/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x2c1184208). One of the two will be used. Which one is undefined.


................................ done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [54]:
template = """
        You are an artificial intelligence assistant who understands insurance.
        Use the following pieces of context to answer the question at the end.
        Extract numbers where appropriate to help answer questions.
        If you don't know the answer, just say you don't know. DO NOT try to make up an answer.

        {context}

        Question: {question}
        Helpful answer:
        """
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [55]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt
    },
    verbose=False,
)

In [53]:
# Use the folling to see the default prompt -- run the above with no chain type kwargs
# print(qa.combine_documents_chain.llm_chain.prompt.template)

## Ask questions ----

In [70]:
%%time
response = qa(
    "Extract the Gross premiums written"
)

CPU times: user 1min 58s, sys: 170 ms, total: 1min 58s
Wall time: 29.7 s


In [71]:
response["result"]

'\n        The gross premiums written are disclosed in the financial statements. They exclude taxes, duties levied on premiums, and adjustments made to premiums during the year for contracts incepting during the financial year together with adjustments made in prior years.'

In [79]:
response["source_documents"][0].metadata["source"]

'docs/hiscox.pdf'

## Single PDF loader -----

In [ ]:
pdf_file = "./data/hiscox.pdf"
pdf_file2 = "/Users/nickaristidou/Downloads/alex_test.pdf"

In [ ]:
images = convert_from_path(pdf_file, dpi=88)

In [ ]:
loader = PyPDFLoader(file_path=pdf_file)
documents = loader.load_and_split()
len(documents)

In [ ]:
loader2 = PyPDFLoader(file_path=pdf_file2)
documents2 = loader2.load_and_split()
len(documents2)

In [ ]:
# print(documents2[5].page_content)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(documents)
len(texts)

In [ ]:
token_splitter = TokenTextSplitter(
                    chunk_size=100, chunk_overlap=10
                )  # This the encoding for text-embedding-ada-002
texts = token_splitter.split_documents(texts)

In [ ]:
texts2 = text_splitter.split_documents(documents2)
len(texts2)

In [ ]:
texts_total = texts + texts2

## Create embeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
db = Chroma.from_documents(texts_total, embeddings, persist_directory="chroma")

## Create Chain

In [ ]:
model_n_ctx = 1000
model_path = "./ggml-gpt4all-j-v1.3-groovy.bin"
llm = GPT4All(model=model_path, backend="gptj", verbose=False)

In [ ]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = config.HF_API

# llm = HuggingFaceHub(
#     repo_id="tiiuae/falcon-7b-instruct",
#     model_kwargs={
#         "temperature":1
#     }
# )

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)

## Ask questions

In [ ]:
%%time
res = qa(
    "What combined ratio did Syndicate 33 report in 2022"
)

In [ ]:
print(res)

In [ ]:
print(res["result"])

In [ ]:
for ref in range(0, len(res["source_documents"])):
    print(res["source_documents"][ref].metadata["page"])

In [ ]:
images[15]

# Ask question about 2nd pdf

In [ ]:
res_2 = qa(
    "Give me a summary of transaction reporting for mifid? extract the response from the text."
)

In [ ]:
# print(res_2)

In [ ]:
res_2["result"]

In [ ]:
res_2["source_documents"]

In [ ]:
test = qa(
    "From page 5 of the hiscox report what was the Gross premiums written in 2022? extract the reponse from the text."
)

In [ ]:
test["result"]

# NOTE: Add prompt and investigate LangchainDirectoryLoader

In [75]:
from tempfile import TemporaryDirectory
from pathlib import Path
import shutil

In [76]:
with TemporaryDirectory() as dir:
        dir
        shutil.copyfileobj("./docs/hiscox.pdf", )
        tmp_path = Path(tmp.name)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 with NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:                                 │
│ ❱ 2 │   │   shutil.copyfileobj("./docs/hiscox.pdf", tmp)                                         │
│   3 │   │   tmp_path = Path(tmp.name)                                                            │
│   4                                                                                              │
│                                                                                                  │
│ /Users/nickaristidou/.pyenv/versions/3.10.8/lib/python3.10/shutil.py:192 in copyfileobj          │
│                                                                                                  │
│    189 │   # Localize variable access to minimize overhead.                                      │
│    190 │   if not length:                                                                        │
│    191 │   │   length = COPY_BUFSIZE                                                             │
│ ❱  192 │   fsrc_read = fsrc.read                                                                 │
│    193 │   fdst_write = fdst.write                                                               │
│    194 │   while True:                                                                           │
│    195 │   │   buf = fsrc_read(length)                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'str' object has no attribute 'read'

In [78]:
Path("./docs/"+"hiscox.pdf")

PosixPath('docs/hiscox.pdf')